In [1]:
import os, sys, glob, obspy
import numpy as np
import pandas as pd

import platform
osname = platform.system()
print(osname)

cwd = os.getcwd()
sys.path.append(os.path.join(cwd, 'campbell'))
#import read_cs_files as campbell

if osname=='Darwin':
    HOME = os.getenv('HOME')
    DROPBOX_TOP = os.path.join(HOME, 'Dropbox')
    SDS_TOP = os.path.join(DROPBOX_TOP, 'DATA', 'SDS')
    WELLDATA_TOP = os.path.join(DROPBOX_TOP, 'DATA', 'KSC', 'KSC_Well_Seismoacoustic_Data/WellData')
    TOB3_DIR = os.path.join(WELLDATA_TOP, 'Uploads')
    PKL_DIR = os.path.join(WELLDATA_TOP, 'Converted')
elif osname=='Linux':
    HOME = os.getenv('HOME')
    DROPBOX_TOP = '/raid/newhome/thompsong/Dropbox'
    SDS_TOP = os.path.join(HOME, 'SDS')
    if not os.path.isdir(SDS_TOP):
        os.mkdir(SDS_TOP)
    WELLDATA_TOP = os.path.join(DROPBOX_TOP, 'DATA', 'KSC', 'KSC_Well_Seismoacoustic_Data/WellData')
    TOB3_DIR = os.path.join(WELLDATA_TOP, 'Uploads')
    PKL_DIR = os.path.join(HOME, 'Converted') 
elif osname=='Windows':
    HOME = 'C:\\Users\\thompsong'
    DROPBOX_TOP = 'D:\\Dropbox'
    SDS_TOP = "D:\\SDS"
    if not os.path.isdir(SDS_TOP):
        os.mkdir(SDS_TOP)
    WELLDATA_TOP = os.path.join(DROPBOX_TOP, 'DATA', 'KSC', 'KSC_Well_Seismoacoustic_Data', 'WellData')
    TOB3_DIR = os.path.join(WELLDATA_TOP, 'Uploads')
    PKL_DIR = 'D:\\Converted' 
lookuptable = os.path.join(PKL_DIR,'lookuptable.csv') 
  
#DROPBOX_PROJECT_DIR = os.path.join(DROPBOX_TOP, 'PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602 Rocket Seismology/202010 KSC Launchpad Erosion')
#EVENT_MSEED_DIR = os.path.join(DROPBOX_TOP, 'DATA', 'KSC', 'KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events')

if not os.path.isdir(PKL_DIR):
    os.mkdir(PKL_DIR)

print(PKL_DIR)

In [2]:
def convert2sds(df, sdsobj, transducersDF): # I think df here is supposed to be from a single picklefile
    #print('***')
    #print(df.columns)  
    #print('***')  
    local_startt = obspy.UTCDateTime(df.iloc[0]['TIMESTAMP'])
    nextt = obspy.UTCDateTime(df.iloc[1]['TIMESTAMP'])
    dt = nextt-local_startt
    utc_startt = localtime2utc(local_startt)
    if utc_startt > obspy.UTCDateTime():
        return
    print('local ', local_startt, '-> UTC ', utc_startt)
    st = obspy.Stream()    
    #print('***')
    #print(df.columns)    
    for col in df.columns[2:]:
        print('Processing column %s' % col)
        this_transducer = transducersDF[(transducersDF['serial']) == col]
        print('***')
        print(this_transducer)
        print('***')
        if len(this_transducer.index)==1:
            this_transducer = this_transducer.iloc[0].to_dict()
            tr = obspy.Trace()
            tr.id = this_transducer['id']
            tr.stats.starttime = utc_startt
            tr.stats.delta = dt  
            tr.data = np.array(df[col])           
            print(f"sampling rate = {tr.stats.sampling_rate}")
            if int(tr.stats.sampling_rate)==20:
                if tr.stats.channel[0]=='H':
                    tr.stats.channel="B%s" % tr.stats.channel[1:]
            if int(tr.stats.sampling_rate)==1:
                if tr.stats.channel[0]=='H' or tr.stats.channel[0]=='B':
                    tr.stats.channel="L%s" % tr.stats.channel[1:]
            #print(tr)
            st.append(tr)
    print('Final Stream object to write')
    print(st)
    sdsobj.stream = st
    successful = sdsobj.write()
    if successful:
        print("Wrote whole Stream object to SDS")
    else: 
        print("Failed to write entire Stream object:")
        print(df)
    return successful


def removed_unnamed_columns(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

def localtime2utc(this_dt):
    hours = 4
    if this_dt>obspy.UTCDateTime(2022,11,6,2,0,0):
        hours = 5
    localTimeCorrection = 3600 * hours
    return this_dt + localTimeCorrection


In [ ]:
sys.path.append(os.path.join(HOME, 'Documents', 'GitHub', 'tremorExplorer', 'lib'))
import SDS
sdsobj = SDS.SDSobj(SDS_TOP, sds_type='D', format='MSEED')
transducersCSVfile = os.path.join(WELLDATA_TOP, 'transducer_metadata.csv')

# Parse lookuptable and convert good CSV files to SDS
lookuptableDF = removed_unnamed_columns(pd.read_csv(lookuptable))
lookuptableDF = lookuptableDF.sort_values(by=['starttime'])
print("lookuptableDF = \n", lookuptableDF.columns)

transducersDF = removed_unnamed_columns(pd.read_csv(transducersCSVfile))
print("transducersDF = \n",transducersDF.columns)

lookuptableDF['SDS'] = False

sds_column_exists = False
if 'SDS' in lookuptableDF.columns:
    sds_column_exists = True

for index, row in lookuptableDF.iterrows():
    print(row)
    row_to_sds_done = False
    if row['passed']:
        if sds_column_exists:
            row_to_sds_done = row['SDS']
        if not row_to_sds_done:
            df2 = pd.read_csv(row['outputfile'])
            print(f"- writing {row['outputfile']} to SDS")
            print("row = \n ",row)
            #print("sdsobj = \n", sdsobj)
            successful = convert2sds(df2, sdsobj, transducersDF)
            lookuptableDF.at[index,'SDS'] = successful
            if successful:
                lookuptableDF.to_csv(lookuptable)
del sdsobj

